20.1 Run Snippets20.1 and 20.2 with timeit.   
Repeat 10 batches of 100 executions.   
What is the minimum elapsed time for each snippet?   

In [1]:
import pandas as pd
import numpy as np
import timeit
from itertools import product
                            
def normal_cartesian():
    ret=[]
    dict0 = {
        'a': ['a1', 'a2', 'a3'],     # 3
        'b': ['b1', 'b2', 'b3'],     # 3
        'c': ['c1', 'c2', 'c3'],     # 3
        'd': ['d1', 'd2', 'd3'],     # 3
        'e': ['e1', 'e2', 'e3'],     # 3
        'f': ['f1', 'f2', 'f3', 'f4']  # 4
    }
    for a in dict0['a']:
        for b in dict0['b']:
            for c in dict0['c']:
                for d in dict0['d']:
                    for e in dict0['e']:
                        for f in dict0['f']:
                            ret.append({'a': a, 'b': b, 'c': c, 'd': d, 'e': e, 'f': f})
    return ret


def vectorized_cartesian():
    dict0 = {
        'a': ['a1', 'a2', 'a3'],     # 3
        'b': ['b1', 'b2', 'b3'],     # 3
        'c': ['c1', 'c2', 'c3'],     # 3
        'd': ['d1', 'd2', 'd3'],     # 3
        'e': ['e1', 'e2', 'e3'],     # 3
        'f': ['f1', 'f2', 'f3', 'f4']  # 4
    }
    jobs=(dict(zip(dict0, x)) for x in product(*dict0.values()))
    ret=[]
    for job in jobs:
        ret.append(job)
    return ret


def timeit_snippet(snippet, batches=10, number=100):
    elapsed_times = []
    for _ in range(batches):
        timer = timeit.Timer(snippet)
        elapsed_time = timer.timeit(number=number)
        elapsed_times.append(elapsed_time)
    return elapsed_times


normal_times = timeit_snippet(lambda: normal_cartesian(), batches=10, number=100)
vectorized_times = timeit_snippet(lambda: vectorized_cartesian(), batches=10, number=100)
print("Min Normal Cartesian Elapsed Times:", min(normal_times))
print("Min Vectorized Cartesian Elapsed Times:", min(vectorized_times))


Min Normal Cartesian Elapsed Times: 0.011711583007127047
Min Vectorized Cartesian Elapsed Times: 0.03308829199522734


20.2 The instructions in Snippet 20.2 are very useful for unit testing, brute force   
searches,and scenario analysis. Can you remember where else in the book have     
you seen them? Where else could they have been used?  

Trading Rule의 threshold 결정할때 상당히 유용하게 사용할 수 있을 것이다.

20.3 Adjust Snippet20.4 to for mmolecules using a two-nested loops scheme,rather     
than a linear scheme.

In [ ]:
from util import multi_thread_barrier_touch, timeit_snippet, get_barrier_touch
import numpy as np

r=np.random.normal(0, 0.01, (1000, 10000))  # Simulated returns
times= timeit_snippet(lambda: multi_thread_barrier_touch(r, width=0.5, num_threads=8, mode="lower_nest"), batches=1, number=10)
print("Min Multi-thread Barrier Touch Elapsed Times:", min(times[0]))

[[1.1040348329988774, 1.1091181250085356, 1.1468629999872064, 1.1127782920084428, 1.1650806250108872, 1.1426218340056948, 1.1234992909885477, 1.092508708999958, 1.4015403749945108, 1.1182131249952363]]
Min Multi-thread Barrier Touch Elapsed Times: 1.092508708999958


20.4 Compare with timeit:  
(a) Snippet20.4,by repeating 10 batches of 100 executions. What is the minimum elapsed time for each snippet?

In [2]:
times= timeit_snippet(lambda: multi_thread_barrier_touch(r, width=0.5, num_threads=8, mode="linear"), batches=10, number=10)
for i in range(10):
    print(f"Batch {i+1} Elapsed Min Time:", min(times[i]))

Batch 1 Elapsed Min Time: 0.8320009580056649
Batch 2 Elapsed Min Time: 0.8241490829968825
Batch 3 Elapsed Min Time: 0.8590347089921124
Batch 4 Elapsed Min Time: 0.8464154589892132
Batch 5 Elapsed Min Time: 0.8303700420074165
Batch 6 Elapsed Min Time: 0.8553720830095699
Batch 7 Elapsed Min Time: 0.8319567500002449
Batch 8 Elapsed Min Time: 0.8608050000038929
Batch 9 Elapsed Min Time: 0.855482833008864
Batch 10 Elapsed Min Time: 0.8785072919999948


(b) Modify Snippet20.4 (from exercise 3),by repeating 10 batches of 100 executions. What is the minimum elapsed time for each snippet?

In [3]:
for mode in ["upper_nest", "lower_nest"]:
    times = timeit_snippet(lambda: multi_thread_barrier_touch(r, width=0.5, num_threads=8, mode=mode), batches=10, number=10)
    for i in range(10):
        print(f"Batch {i+1} Elapsed Min Time for {mode}:", min(times[i]))

Batch 1 Elapsed Min Time for upper_nest: 1.1586178330035182
Batch 2 Elapsed Min Time for upper_nest: 1.1177244999998948
Batch 3 Elapsed Min Time for upper_nest: 1.1159536250052042
Batch 4 Elapsed Min Time for upper_nest: 1.1592967499891529
Batch 5 Elapsed Min Time for upper_nest: 1.1475418750051176
Batch 6 Elapsed Min Time for upper_nest: 1.1848103340016678
Batch 7 Elapsed Min Time for upper_nest: 1.195459166992805
Batch 8 Elapsed Min Time for upper_nest: 1.1846754169964697
Batch 9 Elapsed Min Time for upper_nest: 1.1791347499965923
Batch 10 Elapsed Min Time for upper_nest: 1.1787361669994425
Batch 1 Elapsed Min Time for lower_nest: 1.1925396250007907
Batch 2 Elapsed Min Time for lower_nest: 1.1288349160022335
Batch 3 Elapsed Min Time for lower_nest: 1.1205275829997845
Batch 4 Elapsed Min Time for lower_nest: 1.1444188750028843
Batch 5 Elapsed Min Time for lower_nest: 1.101018291010405
Batch 6 Elapsed Min Time for lower_nest: 1.1578704999992624
Batch 7 Elapsed Min Time for lower_nest: 

20.5 Simplify Snippet 20.4 by using mpPandasObj.

In [4]:
from util import mpPandasObj, get_barrier_touch

price=np.log((1+r).cumprod(axis=0))

times= timeit_snippet(lambda: mpPandasObj(get_barrier_touch, ("price", price), num_threads=8,linear_mols=True,width=0.5), batches=10, number=10)

In [6]:
for i in range(10):
    print(f"Batch {i+1} Elapsed Min Time:", min(times[i]))

Batch 1 Elapsed Min Time: 0.7538561670080526
Batch 2 Elapsed Min Time: 0.7599625000002561
Batch 3 Elapsed Min Time: 0.7493022499984363
Batch 4 Elapsed Min Time: 0.743081250009709
Batch 5 Elapsed Min Time: 0.7602649580076104
Batch 6 Elapsed Min Time: 0.7226747910026461
Batch 7 Elapsed Min Time: 0.7136972080043051
Batch 8 Elapsed Min Time: 0.7105966670060297
Batch 9 Elapsed Min Time: 0.7280133340100292
Batch 10 Elapsed Min Time: 0.7250233340018895


20.6 Modify mpPandasObj to handle the possibility of forming molecules using a
two-nested loops scheme with an upper triangular structure.

In [7]:
times= timeit_snippet(lambda: mpPandasObj(get_barrier_touch, ("price", price), num_threads=8,linear_mols=False,width=0.5), batches=10, number=10)
for i in range(10):
    print(f"Batch {i+1} Elapsed Min Time:", min(times[i]))

Batch 1 Elapsed Min Time: 1.065762458005338
Batch 2 Elapsed Min Time: 1.0408395840058802
Batch 3 Elapsed Min Time: 1.0494476669991855
Batch 4 Elapsed Min Time: 1.0275494999950752
Batch 5 Elapsed Min Time: 1.0626218340039486
Batch 6 Elapsed Min Time: 1.0321894169901498
Batch 7 Elapsed Min Time: 1.0772759579995181
Batch 8 Elapsed Min Time: 1.0512247500009835
Batch 9 Elapsed Min Time: 1.0073582910117693
Batch 10 Elapsed Min Time: 1.0316412920074072
